In [64]:
import numpy as np
import scipy.io
import xarray as xr
from scipy.interpolate import interpn

In [5]:
# Read the sea surface reflectance factors by Mobley
# The header in the file reads as follows :
#  Table of rho values created with FFT surfaces and polarized ray tracing as described in
#    Mobley, C.D., 2015.  Polarized Reflectance and Transmittance Properties of Wind-blown Sea Surfaces,
#    Applied Optics (in review)
#  NOTE: This is a preliminary table for use during review of the above paper.
#    These rho values are for fully developed seas, sun's direct rays parallel to the wind direction,
#    a Rayleigh, single-scattering sky at 550 nm, and wind speeds of 0, 2, 4, 5, 6, 8, 10, 12, 14, and 15 m/s.
#    The final table will be placed online once the above paper has been accepted for publication.
#  rho is computed from the I components of the Stokes radiance vectors: rho = I(surface reflected)/I(sky) [nondimensional]
#  Theta_v is the polar viewing angle measured from 0 at the nadir in degrees.
#  Phi_v is the azimuthal viewing angle measured from the sun at phi_v = 0
#  in degrees.
#  Looping order is
#     wind speed
#        sun zenith angle
#           theta_v
#              phi_v
#  Data blocks of 118 theta_v, phi_v, rho(theta_v, phi_v) records on an (f7.1,f9.1,e15.4) format are separated by 
#   records giving the wind speed and sun zenith angle on an (12x,f5.1,20x,f5.1) format.
#-------------------------------
# Theta_v    Phi_v       rho
#-------------------------------
#WIND SPEED =  0.0; SUN ZENITH ANGLE =  0.0
mat = scipy.io.loadmat('../Data/WaterReflectance/MobleySeaSurfaceRho2015.mat')

In [33]:
# Data is a 4-dimensional array
# Data table is indexed as follows
mat['SeaSurfRho'][0,0][0].shape

(13L, 10L, 10L, 10L)

In [66]:
# Order of axes is :
# Phi-v (azimuthal viewing angle relative to sun in degrees),
# Theta-v (polar viewing angle measured from nadir in degrees)
# SZA (solar zenith angle in degrees)
# Windspeed (metres per second)
mobley_sea_sur_refl = mat['SeaSurfRho'][0,0][0]
Theta_v = np.asarray(mat['SeaSurfRho'][0,0][1], dtype=np.float).squeeze()
Phi_v = np.asarray(mat['SeaSurfRho'][0,0][2], dtype=np.float).squeeze()
SZA = np.asarray(mat['SeaSurfRho'][0,0][3], dtype=np.float).squeeze()
Windspeed = np.asarray(mat['SeaSurfRho'][0,0][4], dtype=np.float).squeeze()
Reference = mat['SeaSurfRho'][0,0][5]
AxisDefinitions = mat['SeaSurfRho'][0,0][6]
AxisOrder = mat['SeaSurfRho'][0,0][7]
AxisUnits = mat['SeaSurfRho'][0,0][8]

In [67]:
Phi_v

array([   0.,   15.,   30.,   45.,   60.,   75.,   90.,  105.,  120.,
        135.,  150.,  165.,  180.])

In [68]:
Theta_v

array([  0. ,  10. ,  20. ,  30. ,  40. ,  50. ,  60. ,  70. ,  80. ,  87.5])

In [69]:
SZA

array([  0. ,  10. ,  20. ,  30. ,  40. ,  50. ,  60. ,  70. ,  80. ,  87.5])

In [70]:
Windspeed

array([  0.,   2.,   4.,   5.,   6.,   8.,  10.,  12.,  14.,  15.])

In [71]:
Reference

array([ u'Mobley, C.D., 2015.  Polarized Reflectance and Transmittance Properties of Wind-blown Sea Surfaces, Applied Optics'], 
      dtype='<U114')

In [72]:
AxisDefinitions

array([[ array([ u'Theta_v is the polar viewing angle measured from 0 at the nadir in degrees'], 
      dtype='<U74'),
        array([u'Phi_v is the azimuthal viewing angle measured from the sun at phi_v = 0'], 
      dtype='<U71'),
        array([u'Solar Zenith Angle'], 
      dtype='<U18'),
        array([u'Wind Speed'], 
      dtype='<U10')]], dtype=object)

In [73]:
AxisOrder

array([[array([u'Phi_v'], 
      dtype='<U5'),
        array([u'Theta_v'], 
      dtype='<U7'),
        array([u'SZA'], 
      dtype='<U3'),
        array([u'WindSpeed'], 
      dtype='<U9')]], dtype=object)

In [61]:
AxisUnits[0][0]

array([u'deg'], 
      dtype='<U3')

In [74]:
# Interpolate at 180 degrees azimuthal viewing angle from sun, Phi_v = 180
# Theta_v = 10 - viewing zenith angle from nadir
# SZA = 60 - solar zenith angle
# Windspeed = 0.5 m/s
reflectance = interpn((Phi_v, Theta_v, SZA, Windspeed), mobley_sea_sur_refl, [180.0, 10.0, 60.0, 0.5])

In [75]:
reflectance

array([ 0.02245075])